# Detecting Building defects using CNN

### Importing Neccessary Libraries

In [5]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


### Image Data Agumentation

In [6]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [7]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory(r'C:\Users\Jayasurya\Desktop\Rock-Classification-Using-CNN-main\train_set',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory(r'C:\Users\Jayasurya\Desktop\Rock-Classification-Using-CNN-main\test_set',
    target_size=(64, 64),batch_size=5,color_mode='rgb',class_mode='categorical') 

Found 700 images belonging to 5 classes.
Found 300 images belonging to 5 classes.


In [8]:
print(x_train.class_indices)#checking the number of classes

{'blue calcite': 0, 'limestone': 1, 'marble': 2, 'olivine': 3, 'red crystal': 4}


In [9]:
print(x_test.class_indices)#checking the number of classes

{'blue calcite': 0, 'limestone': 1, 'marble': 2, 'olivine': 3, 'red crystal': 4}


In [10]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 140, 1: 140, 2: 140, 3: 140, 4: 140})

### Creating the model

In [11]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=5, activation='softmax')) # softmax for more than 2



In [12]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               802944    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6

### Compiling the model

In [13]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [14]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=50, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

C:\Users\Jayasurya\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
140/140 [==============================] - 7s 45ms/step - loss: 1.0150 - accuracy: 0.5571 - val_loss: 0.9312 - val_accuracy: 0.6500
Epoch 2/50
140/140 [==============================] - 5s 32ms/step - loss: 0.6016 - accuracy: 0.7514 - val_loss: 0.6745 - val_accuracy: 0.7267
Epoch 3/50
140/140 [==============================] - 5s 33ms/step - loss: 0.4424 - accuracy: 0.8314 - val_loss: 0.8905 - val_accuracy: 0.5933
Epoch 4/50
140/140 [==============================] - 6s 42ms/step - loss: 0.4195 - accuracy: 0.8457 - val_loss: 0.4955 - val_accuracy: 0.8167
Epoch 5/50
140/140 [==============================] - 8s 58ms/step - loss: 0.3807 - accuracy: 0.8286 - val_loss: 0.5258 - val_accuracy: 0.7500
Epoch 6/50
140/140 [==============================] - 8s 58ms/step - loss: 0.3623 - accuracy: 0.8600 - val_loss: 0.5342 - val_accuracy: 0.8233
Epoch 7/50
140/140 [==============================] - 7s 51ms/step - loss: 0.3078 - accuracy: 0.8771 - val_loss: 0.3230 - val_accuracy: 0.9033

### Saving our model

In [15]:
# Save the model
classifier.save('rockrock.h5')

In [1]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

NameError: name 'classifier' is not defined

### Predicting our results

In [5]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("rock.h5") #loading the model for testing

In [6]:
img = image.load_img(r"C:\Users\Jayasurya\Desktop\Rock-Classification-Using-CNN-main\Sample Inputs\limestone.jpg",
                     grayscale=False,target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict(x)#predicting the classes
pred

array([[0., 0., 1., 0., 0.]], dtype=float32)

In [7]:
import numpy as np
pred=np.argmax(pred,axis=1)

In [8]:
index=['blue calcite', 'limestone', 'marble', 'olivine', 'red crystal']
result=str(index[pred[0]])

result

'marble'

In [9]:
!pip install jupyterthemes

In [3]:
!jt -t  monokai

In [ ]:
!jt -t 